# Web Scraping Part 1: Getting Data from HTML Tables

John McLevey  
February 1, 2018

Today we will work through an example of scraping the web to collect data for research on funding for non-profit organizations. The example below comes from [an article called "Think Tanks, Funding, and the Politics of Policy Knowledge in Canada," which I published in 2014](http://onlinelibrary.wiley.com/doi/10.1111/cars.12033/full). It was based on a chapter from my PhD dissertation (2013). 

The article analyzed funding data for 30 Canadian think tanks over 11 years to weigh in on some pretty high-level debates in sociology and political science about whether or not the politics of any given think tank are associated with the type of funders they have. My analysis was exporatory and, really, just simple description. I collected funding data for all the known think tanks in the country, and then analyzed that data using a really simple quantitative method called [correspondence analysis](https://en.wikipedia.org/wiki/Correspondence_analysis), which is similair to the more widely-known method [principle components analysis](https://en.wikipedia.org/wiki/Principal_component_analysis). You can get the gist of what I found just by reading the article abstract: 

> The relationships between think tanks and their funders are central to theory and public discourse about the politics of policy knowledge, yet very little research systematically examines these relationships across cases. This article evaluates elite, pluralist, and field theories by analyzing original data on funding and politics for 30 think tanks from 2000 to 2011 with comparative and relational methods. I find that foreign donations help support some conservative think tanks, but that it is a small amount of money relative to other funding sources. Domestically, think tank funding is structured by an opposition between donor-funded conservatives and state-funded centrists. Since 2005, the cluster of conservative think tanks funded by private donors has become tighter, while the cluster of think tanks supported by the state has become looser and more reliant on self-generated revenue and interest and investments. These findings cast doubt on predictions derived from elite and pluralist theories, and offer some support for field theory.

The data were scraped from the Charities Listings of the Canada Revenue Agency, which provides Information Returns for all registered charities in the country. Here are a few relevant paragraphs from the article about the data I used: 

> Canadian think tanks have to file annual T3010 Information Returns with the CRA to maintain their tax-exempt charitable status. The returns are available in a searchable database, making it is possible to look up information for specific organizations at specific times. To evaluate claims about funding and political orientations, I collected financial data from 2000 to 2011, or from founding date to latest filing, for 30 think tanks. To automate the collection process and eliminate human error, I used a Perl script to scrape data from the charities listings for each organization and create a data set of all publicly available information.

> The case-selection process was inclusive. I compiled a list of think tanks mentioned by interviewees, from the literature, news stories and op-eds, and public lists and rankings (e.g., McGann 2010). I excluded university-based institutes, consulting firms, some very small single-issue think tanks, and one nonprofit think tank without charitable status. Finally, I included two organizations whose activities are more extensive than others in the analysis: the David Suzuki Foundation and the Institute of Marriage and Family.

> Under “Section E: Financial Information” of the Information Returns, I combined municipal, provincial, and federal funding into “state funding,” and I classified rental income, memberships, dues, fees, and sale of goods and services (except to government) as “self-generated revenue.” I combined entries for tax-receipted and non-tax-receipted gifts, and gifts from other registered charities, as “private donations.”

> ...

Today, we are going to start the process of developing a web scraper to collect similair data from the [CRA Charities Listing](http://www.cra-arc.gc.ca/ebci/haip/srch/advancedsearch-eng.action). This is *not* the scraper I used to collect the data used in the journal article. That scraper was written in [Perl](https://www.perl.org) by my friend Andrew Osmond. 

![XKCD on Perl and regular expressions...](https://imgs.xkcd.com/comics/perl_problems.png)

Most of this notebook will use the Montréal-based think tank [IRPP (the Institute for Research on Public Policy / Institut de recherche en politiques publiques)](http://irpp.org) as an example, but you can use any non-profit you like. Just find them in the listing and make the necessary modifications to the code below. 

Note that we don't actually need to write a lot of code to get what we want. The heavy lifting is really being done by [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and [Pandas](http://pandas.pydata.org). We will make some improvements to this scraper in subsequent classes. 

# Getting Started

This is where we start: http://www.cra-arc.gc.ca/ebci/haip/srch/advancedsearch-eng.action

![](images/Screenshot 2018-02-01 09.10.26.png)

Search for "Institute for Research on Public Policy." The correct result is the second one. Montréal, founded in 1972. Select it. 

We want to look at the full view, **not quick view**, of the 2017 Information Return. 

![](images/Screenshot 2018-02-01 09.12.45.png)

Once you are in the full view, you will see that you have still more options to choose from. We are going to pick "Schedule D: Financial Information," and then "Schedule 6: Detailed Financial Information." This is the page you should find yourself on: 

![](images/Screenshot 2018-02-01 09.15.12.png)

There are sections you can expand: 

1. Assets
2. Liabilities
3. Revenue
4. Expenditures

In Firefox or Chrome, inspect the source for this page and learn what you can about how it is structured. 

![](images/Screenshot 2018-02-01 09.17.03.png)

# Developing a Scraper

Let's explore multiple options for scraping html tables. Here, we will use Pandas. Another option, which I encourage you to explore, is to use Beautiful Soup. 

In [10]:
irpp_2017 = 'http://www.cra-arc.gc.ca/ebci/haip/srch/t3010form23sched6-eng.action?b=118969393RR0001&fpe=2017-03-31&n=INSTITUTE+FOR+RESEARCH+ON+PUBLIC+POLICY+L%27INSTITUT+DE+RECHERCHE+EN+POLITIQUES+PUBLIQUES&r=http%3A%2F%2Fwww.cra-arc.gc.ca%3A80%2Febci%2Fhaip%2Fsrch%2Ft3010form23-eng.action%3Fb%3D118969393RR0001%26amp%3Bfpe%3D2017-03-31%26amp%3Bn%3DINSTITUTE%2BFOR%2BRESEARCH%2BON%2BPUBLIC%2BPOLICY%2BL%2527INSTITUT%2BDE%2BRECHERCHE%2BEN%2BPOLITIQUES%2BPUBLIQUES'

## Option 1: Pandas

Scraping data from an HTML table is *easy* if we let Pandas do the heavy lifting. The [`html_table()` method]() does most of what we want. It will accept a url, look for html tables on the page, create dataframes from those html tables, and return the tables to you in a list. If there is 1 table on the webpage, there will be one dataframe in the list and you can access it by refernecing the index (which is 0). If there are four tables, there will be four dataframes in the list. You get the point...

In [35]:
import os
import pandas as pd

In [14]:
df_list = pd.read_html(irpp_2017)
len(df_list)

4

There are four dataframes in our list of dataframes. Couldn't be easier. 

Let's look at the table with information on revenue. It's the third table, index 2. 

In [17]:
rev = df_list[2]
rev

,Description of revenue,Line number,Amount
0,Total eligible amount of all gifts for which t...,4500,"$ 29,155"
1,Total eligible amount of tax-receipted tuition...,5610,n/a
2,Total amount of 10 year gifts received,4505,n/a
3,Total amount received from other registered ch...,4510,n/a
4,Total other gifts received for which a tax rec...,4530,n/a
5,Total revenue received from federal government,4540,$ 300
6,Total revenue received from provincial/​territ...,4550,n/a
7,Total revenue received from municipal/​regiona...,4560,n/a
8,Total tax-receipted revenue from all sources o...,4571,n/a
9,Total non tax-receipted revenue from all sourc...,4575,n/a


This looks pretty good. But there are `$` and `,` in the values of the `Amount` column. The values are strings not numbers. Also, there a lot of `n/a` values. 

How can we clean this up? One option is to remove the `$` and `,` characters and convert the numbers to integers or floats. We can do this using the [`.replace()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.replace.html) from Pandas and simple regular expression. For example:

In [18]:
rev['Amount'] = rev['Amount'].replace('[\$,]', '', regex=True)
rev

,Description of revenue,Line number,Amount
0,Total eligible amount of all gifts for which t...,4500,29155
1,Total eligible amount of tax-receipted tuition...,5610,n/a
2,Total amount of 10 year gifts received,4505,n/a
3,Total amount received from other registered ch...,4510,n/a
4,Total other gifts received for which a tax rec...,4530,n/a
5,Total revenue received from federal government,4540,300
6,Total revenue received from provincial/​territ...,4550,n/a
7,Total revenue received from municipal/​regiona...,4560,n/a
8,Total tax-receipted revenue from all sources o...,4571,n/a
9,Total non tax-receipted revenue from all sourc...,4575,n/a


This looks OK, but actually there are still some issues with it. The `Amount` values look cleaner than they actually are. Jupyter is hiding things from us! If we were to write this dataframe to file and open it up again, we would find some strange characters. They have to be removed. 

And what should we do about these `n/a` values? Python is interpreting them as strings (which they are). 

In the context of these Information Returns, what does `n/a` really mean? 

It is not likely to be "missing data," comparable to a survey question that a participant skips because they don't want to answer the question. Why not? Non-profits can't simply decline to answer questions on their Information Returns for the CRA. That might not go over so well.  

Should we just set these values to 0? After all, one way of interpreting an `n/a` response to `Total amount of 10 year gifts received` (Line 4505) is that the organization recieved 0 ten year gifts. But 0 and not applicable are not really the same thing. So let's not treat it as missing data. 

Instead, let's strip out anything from the amount column that is not a number or a `.`. As far as we know, there are no `.` in these tables, but if one ever showed up unexpectedly, we would *not* want to remove it. 

We can do this with a bit of clever regex. In the context of the `replace()` method below, `[^\d\^.]` will enable us to strip anything that is not a number `.`. 

In [19]:
rev['Amount'] = rev['Amount'].replace('[^\d^\.]', '', regex=True)

In [20]:
rev

,Description of revenue,Line number,Amount
0,Total eligible amount of all gifts for which t...,4500,29155
1,Total eligible amount of tax-receipted tuition...,5610,
2,Total amount of 10 year gifts received,4505,
3,Total amount received from other registered ch...,4510,
4,Total other gifts received for which a tax rec...,4530,
5,Total revenue received from federal government,4540,300
6,Total revenue received from provincial/​territ...,4550,
7,Total revenue received from municipal/​regiona...,4560,
8,Total tax-receipted revenue from all sources o...,4571,
9,Total non tax-receipted revenue from all sourc...,4575,


Looks better. 

We can generalize this solution with a function. We will include an optional argument to clean the `Amount` columns in the dataframes. It will be off by default. 

In [28]:
def scrape_financial_tables(url, org, year, clean = False):
    """
    Scrapes html tables from a 'Schedule 6: Detailed financial information' page from the Canada Revenue Agency Charities Listings. 
    Returns a list of Pandas dataframes.
    """
    dfs = pd.read_html(url)
    if clean == True:
        for d in dfs:
            d['Amount'] = d['Amount'].replace('[^\d^\.]', '', regex=True)
    return dfs

In [29]:
unclean = scrape_financial_tables(irpp_2017, 'IRPP', 2017)
unclean[0]

,Description of assets,Line number,Amount
0,"Cash, bank accounts, and short-term investments",4100,"$ 161,865"
1,Amounts receivable from non-arm's length persons,4110,n/a
2,Amounts receivable from all others,4120,"$ 391,716"
3,Investments in non-arm's length persons,4130,n/a
4,Long-term investments,4140,"$ 43,028,613"
5,Inventories,4150,n/a
6,Land and buildings in Canada,4155,n/a
7,Other capital assets in Canada,4160,"$ 550,670"
8,Capital assets outside Canada,4165,n/a
9,Accumulated amortization of capital assets,4166,"$ -535,802"


In [30]:
clean = scrape_financial_tables(irpp_2017, 'IRPP', 2017, clean = True)
clean[0]

,Description of assets,Line number,Amount
0,"Cash, bank accounts, and short-term investments",4100,161865
1,Amounts receivable from non-arm's length persons,4110,
2,Amounts receivable from all others,4120,391716
3,Investments in non-arm's length persons,4130,
4,Long-term investments,4140,43028613
5,Inventories,4150,
6,Land and buildings in Canada,4155,
7,Other capital assets in Canada,4160,550670
8,Capital assets outside Canada,4165,
9,Accumulated amortization of capital assets,4166,535802


Finally, let's write our new data to disk. First, if there is no data subdirectory, create one. 

In [42]:
if os.path.isdir('data/') == False:
    os.mkdir('data')

In [44]:
clean[0].to_csv('data/irpp_2017_assets.csv')
clean[1].to_csv('data/irpp_2017_liabilities.csv')
clean[2].to_csv('data/irpp_2017_revenue.csv')
clean[3].to_csv('data/irpp_2017_expenditures.csv')

# Your Turn!

Work on one (or more) of the following: 

1. Modify the functions developed above to incorporate some error handling. 
2. Grab data for other years by manipulating the url strings. (Crawling comes later...)

More coming later...